In [ ]:
# Qwen2.5-Coder-1.5B

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Coder-1.5B-Instruct", trust_remote_code=True)

# Load model with memory optimization using bfloat16 precision
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-Coder-1.5B-Instruct", 
    trust_remote_code=True, 
    torch_dtype=torch.bfloat16
).cuda()

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Assume tokenizer and model are already loaded
# tokenizer = AutoTokenizer.from_pretrained("some-chat-model")
# model = AutoModelForCausalLM.from_pretrained("some-chat-model")

messages = [{'role': 'user', 'content': "Write a Python function of bubble sort"}]

# Apply chat template and move to device
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

# Handle different possible return types
if isinstance(inputs, torch.Tensor):
    # If inputs is a tensor, assume it's input_ids
    current_input = inputs
    # Create attention_mask (all ones) if not provided
    attention_mask = torch.ones_like(current_input, device=current_input.device)
elif isinstance(inputs, dict):
    # If inputs is a dictionary, access input_ids and attention_mask
    current_input = inputs['input_ids']
    attention_mask = inputs.get('attention_mask', torch.ones_like(current_input, device=current_input.device))
else:
    raise ValueError(f"Unexpected return type from apply_chat_template: {type(inputs)}")

# Ensure tensors are 2D (batch_size, sequence_length)
if current_input.dim() == 1:
    current_input = current_input.unsqueeze(0)
    attention_mask = attention_mask.unsqueeze(0)
elif current_input.dim() != 2:
    raise ValueError(f"input_ids must be 1D or 2D, got {current_input.dim()}D")

# Proceed with generation
generated_tokens = []

print("Generating tokens:")
while len(generated_tokens) < 2048:
    outputs = model.generate(
        input_ids=current_input,
        attention_mask=attention_mask,
        max_new_tokens=1,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
    next_token = outputs[0, -1]  # Last token in the sequence (batch_size=1)
    generated_tokens.append(next_token)

    if next_token == tokenizer.eos_token_id:
        break

    # Append the new token to the input sequence
    next_token_tensor = next_token.unsqueeze(0).unsqueeze(0)  # Shape (1, 1)
    current_input = torch.cat([current_input, next_token_tensor], dim=1)
    attention_mask = torch.cat([attention_mask, torch.ones((1, 1), dtype=attention_mask.dtype, device=attention_mask.device)], dim=1)

    # Print the decoded token
    print(tokenizer.decode(next_token, skip_special_tokens=True), end=' ')

# Decode and print the final generated code
generated_code = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print("\n\nFinal generated code:\n")
print(generated_code)

Generating tokens:
   
  
 ``` python 
 def  bubble _sort (arr ):
      n  =  len (arr )
     
      #  Traverse  through  all  array  elements 
      for  i  in  range (n ):
          #  Last  i  elements  are  already  in  place 
          for  j  in  range ( 0 ,  n -i - 1 ):
              #  Traverse  the  array  from   0  to  n -i - 1 
              #  Swap  if  the  element  found  is  greater  than  the  next  element 
              if  arr [j ]  >  arr [j + 1 ]:
                  arr [j ],  arr [j + 1 ]  =  arr [j + 1 ],  arr [j ]
                 
      return  arr 

 #  Example  usage :
 arr  =  [ 6 4 ,   3 4 ,   2 5 ,   1 2 ,   2 2 ,   1 1 ,   9 0 ]
 print (" Sorted  array :",  bubble _sort (arr ))
 `` `

 This  Python  function  implements  the  bubble  sort  algorithm ,  which  repeatedly  steps  through  the  list ,  compares  adjacent  elements  and  swaps  them  if  they  are  in  the  wrong  order .  The  pass  through  the  list  is  repeated  until  the  list  is  sor